# MongoDB Assg

submitted by: Syed Adeel Abdullah


In [4]:
import ijson
import pandas as pd
from pymongo import MongoClient

In [5]:
filename = "rows.json"

In [6]:
with open(filename, 'r') as f:
    objects = ijson.items(f, 'meta.view.columns.item')
    columns = list(objects)

In [7]:
print(columns[0])

{'renderTypeName': 'meta_data', 'dataTypeName': 'meta_data', 'format': {}, 'name': 'sid', 'flags': ['hidden'], 'id': -1, 'position': 0, 'fieldName': ':sid'}


In [8]:
column_names = [col["fieldName"] for col in columns]

In [10]:
#Only important columns
important_columns = [
    "date_of_stop", 
    "color", 
    "arrest_type"
]

In [11]:
data = []
with open(filename, 'r') as f:
    objects = ijson.items(f, 'data.item')
    for row in objects:
        selected_row = []
        for item in important_columns:
            selected_row.append(row[column_names.index(item)])
        data.append(selected_row)

In [31]:
df_TV = pd.DataFrame(data, columns=important_columns)

In [32]:
df_TV['day_of_stop'] = pd.to_datetime(df_TV['date_of_stop'], errors='coerce')

In [33]:
df_TV['day_of_stop']=df_TV['day_of_stop'].dt.weekday_name

In [41]:
mongo_db = client['tf_vn']


In [42]:
mongo_db.collections.insert_many(df_TV.to_dict('records'))

In [43]:
StopByColor=mongo_db.collections.aggregate([{"$group" : {"_id":"$color", "count":{"$sum":1}}}])

In [44]:
#How many stops are made by car color
for stop in StopByColor:
    print (stop)

{'_id': 'CHROME', 'count': 23}
{'_id': 'COPPER', 'count': 357}
{'_id': 'YELLOW', 'count': 4247}
{'_id': 'BRONZE', 'count': 2588}
{'_id': 'BROWN', 'count': 5245}
{'_id': 'GREEN, LGT', 'count': 6491}
{'_id': 'MAROON', 'count': 19330}
{'_id': 'PURPLE', 'count': 2211}
{'_id': 'GOLD', 'count': 35901}
{'_id': 'PINK', 'count': 167}
{'_id': 'CREAM', 'count': 746}
{'_id': 'TAN', 'count': 23864}
{'_id': 'GREEN, DK', 'count': 12812}
{'_id': 'BLUE, LIGHT', 'count': 14942}
{'_id': 'GREEN', 'count': 43944}
{'_id': 'ORANGE', 'count': 3829}
{'_id': 'WHITE', 'count': 164915}
{'_id': 'CAMOUFLAGE', 'count': 20}
{'_id': 'MULTICOLOR', 'count': 924}
{'_id': 'BLUE', 'count': 80581}
{'_id': 'SILVER', 'count': 199971}
{'_id': 'N/A', 'count': 12508}
{'_id': 'GRAY', 'count': 116635}
{'_id': 'BLUE, DARK', 'count': 23227}
{'_id': 'RED', 'count': 86613}
{'_id': None, 'count': 1746}
{'_id': 'BLACK', 'count': 216292}
{'_id': 'BEIGE', 'count': 13160}


In [45]:
df_TV["color"].value_counts() #verify with pandas


BLACK          216292
SILVER         199971
WHITE          164915
GRAY           116635
RED             86613
BLUE            80581
GREEN           43944
GOLD            35901
TAN             23864
BLUE, DARK      23227
MAROON          19330
BLUE, LIGHT     14942
BEIGE           13160
GREEN, DK       12812
N/A             12508
GREEN, LGT       6491
BROWN            5245
YELLOW           4247
ORANGE           3829
BRONZE           2588
PURPLE           2211
MULTICOLOR        924
CREAM             746
COPPER            357
PINK              167
CHROME             23
CAMOUFLAGE         20
Name: color, dtype: int64

In [46]:
#Find the value counts of arrest type
value_counts=mongo_db.collections.aggregate([{"$group" : {"_id":"$arrest_type", "count":{"$sum":1}}}])

In [47]:
for counts in value_counts:
    print (counts)

{'_id': 'K - Aircraft Assist', 'count': 44}
{'_id': 'J - Unmarked Moving Radar (Moving)', 'count': 299}
{'_id': 'P - Mounted Patrol', 'count': 260}
{'_id': 'H - Unmarked Moving Radar (Stationary)', 'count': 561}
{'_id': 'I - Marked Moving Radar (Moving)', 'count': 1545}
{'_id': 'C - Marked VASCAR', 'count': 399}
{'_id': 'D - Unmarked VASCAR', 'count': 232}
{'_id': 'E - Marked Stationary Radar', 'count': 6882}
{'_id': 'S - License Plate Recognition', 'count': 12875}
{'_id': 'M - Marked (Off-Duty)', 'count': 1632}
{'_id': 'R - Unmarked Laser', 'count': 5083}
{'_id': 'N - Unmarked (Off-Duty)', 'count': 161}
{'_id': 'L - Motorcycle', 'count': 10336}
{'_id': 'Q - Marked Laser', 'count': 110653}
{'_id': 'G - Marked Moving Radar (Stationary)', 'count': 4077}
{'_id': 'B - Unmarked Patrol', 'count': 35195}
{'_id': 'O - Foot Patrol', 'count': 11397}
{'_id': 'F - Unmarked Stationary Radar', 'count': 772}
{'_id': 'A - Marked Patrol', 'count': 890886}


In [48]:
df_TV["arrest_type"].value_counts() # verify with pandas

A - Marked Patrol                         890886
Q - Marked Laser                          110653
B - Unmarked Patrol                        35195
S - License Plate Recognition              12875
O - Foot Patrol                            11397
L - Motorcycle                             10336
E - Marked Stationary Radar                 6882
R - Unmarked Laser                          5083
G - Marked Moving Radar (Stationary)        4077
M - Marked (Off-Duty)                       1632
I - Marked Moving Radar (Moving)            1545
F - Unmarked Stationary Radar                772
H - Unmarked Moving Radar (Stationary)       561
C - Marked VASCAR                            399
J - Unmarked Moving Radar (Moving)           299
P - Mounted Patrol                           260
D - Unmarked VASCAR                          232
N - Unmarked (Off-Duty)                      161
K - Aircraft Assist                           44
Name: arrest_type, dtype: int64

In [49]:
#Find which days result in the most traffic stop
day_stop=mongo_db.collections.aggregate([{"$group" : {"_id":"$day_of_stop", "count":{"$sum":1}}},{"$sort":{"count":-1}}]) 

In [50]:
for day_week in day_stop:
    if (day_week["_id"])!=None:
        print (day_week)

{'_id': 'Tuesday', 'count': 187270}
{'_id': 'Wednesday', 'count': 176103}
{'_id': 'Friday', 'count': 173063}
{'_id': 'Thursday', 'count': 168624}
{'_id': 'Monday', 'count': 147816}
{'_id': 'Saturday', 'count': 127443}
{'_id': 'Sunday', 'count': 112970}


In [51]:
df_TV['day_of_stop'].value_counts() # verify with pandas

Tuesday      187270
Wednesday    176103
Friday       173063
Thursday     168624
Monday       147816
Saturday     127443
Sunday       112970
Name: day_of_stop, dtype: int64

In [52]:
client.close()